## Run all the following code.
## Output will be output_icon.png

In [24]:
import os
import requests
# pip install openai
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(
    # set openai API key to an environment variable, then replace "OPENAI_API_KEY" with the name of the environment variable.
    # IMPORTANT!! Don't put actual API key here and push to the Github.
    api_key=os.environ.get("OPENAI_API_KEY_DEWAN")
)

# modify object name here. Use simple words.
objectName = "round arrow"

# Generate the image
response = client.images.generate(
  model="dall-e-3",
  prompt=f"You are the world best icon designer. Create a simple white {objectName} icon for me. The icon content should be half of the image. The rest of the image should be black. Do not include anything else in the image except the icon itself. The icon should be one entity. There will only be black and white in the generated image.",
  size="1024x1024",
  quality="standard",
  n=1,
)

# Get the image URL from the response
image_url = response.data[0].url

# Download the image
image_response = requests.get(image_url)

# Save the image locally
if image_response.status_code == 200:
    with open("generated_image.png", "wb") as file:
        file.write(image_response.content)
else:
    print("Failed to download image")


In [ ]:
# !pip install pillow
from PIL import Image

def change_non_white_to_transparent_and_resize(image_path, output_path, threshold=200, size=(56, 56)):
    # Open the image
    image = Image.open(image_path).convert("RGBA")
    
    # Get data
    data = image.getdata()
    
    new_data = []
    for item in data:
        # Change pixels not close to white to transparent
        if all(channel > threshold for channel in item[:3]):
            new_data.append(item)
        else:
            new_data.append((255, 255, 255, 0))  # White and transparent
    
    # Update image data
    image.putdata(new_data)
    
    # Resize the image
    image = image.resize(size, Image.ANTIALIAS)
    
    # Save the image
    image.save(output_path, "PNG")

# Example usage
change_non_white_to_transparent_and_resize('generated_image.png', 'output_icon.png')


In [26]:
from PIL import Image
import base64
import svgwrite

def png_to_svg_with_transparency(png_path, svg_path):
    # Open the image
    image = Image.open(png_path).convert("RGBA")
    
    # Save the image as a PNG in memory
    with open(png_path, "rb") as img_file:
        b64_string = base64.b64encode(img_file.read()).decode('utf-8')

    # Create an SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile='full')
    
    # Embed the PNG image with base64 encoding
    dwg.add(dwg.image(href=f'data:image/png;base64,{b64_string}', insert=(0, 0), size=image.size))
    
    # Save the SVG file
    dwg.save()

# Example usage
png_to_svg_with_transparency('output_icon.png', 'output_icon.svg')